In [7]:
# Import Library
import pandas as pd
import glob
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm


c:\Users\Jue\Desktop\NLP_Final_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Directory where your files are located
data_dir = "data"

# Find all the final CSVs in the folder
file_paths = glob.glob(os.path.join(data_dir, "tarot_readings_*_final.csv"))

# Empty list to store DataFrames
dataframes = []

# Loop through files and load them with category name
for file_path in file_paths:
    df = pd.read_csv(file_path)
    
    # Extract category from filename
    filename = os.path.basename(file_path)
    category = filename.split('_')[2]  # Assumes the format tarot_readings_<category>_final.csv
    df["category"] = category
    
    dataframes.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

#drop the category column
df = combined_df.drop('category', axis=1)

# Save to a new CSV
df.to_csv("tarot_readings_combined.csv", index=False)

print("All files combined and saved as 'tarot_readings_combined.csv'")


All files combined and saved as 'tarot_readings_combined.csv'


In [6]:
df.columns

Index(['zodiac', 'question', 'card1', 'card2', 'card3', 'reading'], dtype='object')

In [8]:
# Load 3-class sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Sentiment labels
labels = ['negative', 'neutral', 'positive']

# Sentiment function
def get_sentiment(text):
    try:
        # Tokenize
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        # Get model output
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            pred = torch.argmax(probs, dim=1).item()
        return labels[pred]
    except Exception as e:
        return "neutral"  # fallback for any errors

c:\Users\Jue\Desktop\NLP_Final_Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jue\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fal

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [10]:
# Load your dataset
df = pd.read_csv("tarot_readings_combined.csv")  # Adjust path if needed


In [11]:
# Run sentiment analysis with progress bar
tqdm.pandas()

df["question_sentiment"] = df["question"].progress_apply(get_sentiment)
df["reading_sentiment"] = df["reading"].progress_apply(get_sentiment)


100%|██████████| 11990/11990 [12:27<00:00, 16.05it/s]


In [12]:
df.to_csv("tarot_readings_with_sentiments.csv", index=False)
print("✅ Sentiment analysis complete! Saved as 'tarot_readings_with_sentiments.csv'")

✅ Sentiment analysis complete! Saved as 'tarot_readings_with_sentiments.csv'
